In [9]:
import json
import numpy as np
import wowsims

This example loads simulation settings for a fearl druid.
These are stored in a json file that is included in the repository.

In [10]:
f = open('data/feral.json')
settings = json.load(f)

The easiest way to reset a simulation, for now, is to create a new one.
To do that, the settings json is encoded as a utf-8 string and passed as input to `new`.

In [11]:
def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))

The sim library currently ignores the iteration parameter in the settings to better support interactive mode (see a few cells down).
The following code is an example of how to run multiple iterations in this environment.

In [12]:
iterations = settings['simOptions']['iterations']
duration = settings['encounter']['duration']

In [13]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 10548.091761180927


Interactive mode is a way to provide input to the sim while it is running.
This can be used, for example, as a way to test rotation ideas.
Because interactive mode is enabled, the code below does nothing other than auto attack.

In [14]:
settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 2180.8624313585433


The next run uses the shred spell when it can.
Spells are accesed via the casting character's spellbook.
In this case, we have a help file `feral.py` that defines some globals for the spells we are interested in.

In [15]:
from feral import Spells

Spells.register()

settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Shred)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 4465.983277973895


For more complex input logic having some context would be helpful.
The `feral.py` file also contains some aura labels and registers them with the sim. 

In [16]:
from feral import Auras, TargetAuras

Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            Auras.update()
            TargetAuras.update()
            eng = wowsims.getEnergy()
            cp = wowsims.getComboPoints()
            if cp > 0 and Auras.get_dur('Savage Roar Aura') <= 0.5:
                wowsims.trySpell(Spells.Roar)
            elif cp < 5:
                wowsims.trySpell(Spells.Shred)
            else:
                wowsims.trySpell(Spells.Bite)

    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 6196.483754958636


The examples so far have only looked at damage output.
More detailed spell metrics are also available.

In [19]:
settings['simOptions']['interactive'] = False

while not wowsims.step():
    pass
json_string = wowsims.getSpellMetrics()
cast_metrics = json.loads(json_string)
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    print(f'{spell_id}: {metrics[0]}')

0: {'Casts': 269, 'Misses': 6, 'Hits': 35, 'Crits': 169, 'Crushes': 0, 'Dodges': 0, 'Glances': 59, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 535870.7472940217, 'TotalThreat': 380468.23057875555, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 0}
48572: {'Casts': 48, 'Misses': 0, 'Hits': 24, 'Crits': 24, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 346533.49303743814, 'TotalThreat': 246038.78005658096, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 48000000000}
48577: {'Casts': 9, 'Misses': 0, 'Hits': 2, 'Crits': 7, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 171809.8370161166, 'TotalThreat': 121984.98428144275, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime': 9000000000}
52610: {'Casts': 9, 'Misses': 0, 'Hits': 0, 'Crits': 0, 'Crushes': 0, 'Dodges': 0, 'Glances': 0, 'Parries': 0, 'Blocks': 0, 'TotalDamage': 0, 'TotalThreat': 0, 'TotalHealing': 0, 'TotalShielding': 0, 'TotalCastTime':